# Word Count

### Counting the number of occurances of words in a text is a popular first exercise using map-reduce.

## The Task
**Input:** A text file consisisting of words separated by spaces.  
**Output:** A list of words and their counts, sorted from the most to the least common.

We will use the book "Moby Dick" as our input.

In [1]:
#start the SparkContext
from pyspark import SparkContext
sc=SparkContext(master="local[4]")

22/10/10 10:41:32 WARN Utils: Your hostname, HP-G62 resolves to a loopback address: 127.0.1.1; using 192.168.18.113 instead (on interface enp3s0)
22/10/10 10:41:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/10 10:41:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/10 10:41:35 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/10/10 10:41:35 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/10/10 10:41:35 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
22/10/10 10:41:35 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
22/10/10 10:41:35 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.


### Setup a plan for pretty print

In [3]:
def pretty_print_plan(rdd):
    for x in rdd.toDebugString().decode().split('\n'):
        print(x)

### Use `textFile()` to read the text

In [4]:
%%time
text_file = sc.textFile("Moby-Dick.txt")

CPU times: user 6.2 ms, sys: 0 ns, total: 6.2 ms
Wall time: 1.19 s


In [5]:
type(text_file)

pyspark.rdd.RDD

## Steps for counting the words

* split line by spaces.
* map `word` to `(word,1)`
* count the number of occurances of each word.

In [6]:
%%time
words =text_file.flatMap(lambda line: line.split(" "))
not_empty = words.filter(lambda x: x!='') 
key_values= not_empty.map(lambda word: (word, 1)) 
key_values.collect()
counts=key_values.reduceByKey(lambda a, b: a + b)

CPU times: user 202 ms, sys: 16.9 ms, total: 219 ms
Wall time: 3.46 s


In [12]:
counts.take(50)

[('The', 549),
 ('Project', 79),
 ('EBook', 1),
 ('of', 6587),
 ('Moby', 79),
 ('is', 1586),
 ('use', 35),
 ('anyone', 5),
 ('anywhere', 11),
 ('at', 1227),
 ('no', 447),
 ('restrictions', 2),
 ('whatsoever.', 5),
 ('may', 223),
 ('it,', 237),
 ('give', 68),
 ('away', 117),
 ('re-use', 2),
 ('this', 1169),
 ('online', 4),
 ('www.gutenberg.org', 2),
 ('Author:', 1),
 ('Last', 1),
 ('January', 1),
 ('3,', 2),
 ('Posting', 1),
 ('Date:', 2),
 ('#2701]', 1),
 ('June,', 3),
 ('Language:', 1),
 ('English', 42),
 ('***', 6),
 ('OF', 59),
 ('THIS', 12),
 ('GUTENBERG', 3),
 ('MOBY', 3),
 ('DICK;', 3),
 ('Lazarus', 6),
 ('Original', 1),
 ('combination', 2),
 ('now-defunct', 1),
 ('project', 3),
 ('version', 3),
 ('are', 586),
 ('University', 1),
 ('Adelaide', 1),
 ('preserving', 3),
 ('version.', 1),
 ('resulting', 3),
 ('was', 1566)]

### flatMap()
Note the line:
```python
words =     text_file.flatMap(lambda line: line.split(" "))
```
Why are we using `flatMap`, rather than `map`?

The reason is that the operation `line.split(" ")` generates a **list** of strings, so had we used `map` the result would be an RDD of lists of words. Not an RDD of words.

The difference between `map` and `flatMap` is that the second expects to get a list as the result from the map and it **concatenates** the lists to form the RDD.

## The execution plan
In the last cell we defined the execution plan, but we have not started to execute it.

* Preparing the plan took ~100ms, which is a non-trivial amount of time, 
* But much less than the time it will take to execute it.
* Lets have a look a the execution plan.

### Understanding the details
To see which step in the plan corresponds to which RDD we print out the execution plan for each of the RDDs.  

Note that the execution plan for `words`, `not_empty` and `key_values` are all the same.

In [29]:
pretty_print_plan(text_file)

(2) Data/Moby-Dick.txt MapPartitionsRDD[7] at textFile at NativeMethodAccessorImpl.java:0 []
 |  Data/Moby-Dick.txt HadoopRDD[6] at textFile at NativeMethodAccessorImpl.java:0 []


In [31]:
pretty_print_plan(words)

(2) PythonRDD[23] at RDD at PythonRDD.scala:53 []
 |  Data/Moby-Dick.txt MapPartitionsRDD[7] at textFile at NativeMethodAccessorImpl.java:0 []
 |  Data/Moby-Dick.txt HadoopRDD[6] at textFile at NativeMethodAccessorImpl.java:0 []


In [32]:
pretty_print_plan(not_empty)

(2) PythonRDD[24] at RDD at PythonRDD.scala:53 []
 |  Data/Moby-Dick.txt MapPartitionsRDD[7] at textFile at NativeMethodAccessorImpl.java:0 []
 |  Data/Moby-Dick.txt HadoopRDD[6] at textFile at NativeMethodAccessorImpl.java:0 []


In [33]:
pretty_print_plan(key_values)

(2) PythonRDD[18] at collect at <timed exec>:4 []
 |  Data/Moby-Dick.txt MapPartitionsRDD[7] at textFile at NativeMethodAccessorImpl.java:0 []
 |  Data/Moby-Dick.txt HadoopRDD[6] at textFile at NativeMethodAccessorImpl.java:0 []


In [34]:
pretty_print_plan(counts)

(2) PythonRDD[25] at RDD at PythonRDD.scala:53 []
 |  MapPartitionsRDD[22] at mapPartitions at PythonRDD.scala:145 []
 |  ShuffledRDD[21] at partitionBy at NativeMethodAccessorImpl.java:0 []
 +-(2) PairwiseRDD[20] at reduceByKey at <timed exec>:5 []
    |  PythonRDD[19] at reduceByKey at <timed exec>:5 []
    |  Data/Moby-Dick.txt MapPartitionsRDD[7] at textFile at NativeMethodAccessorImpl.java:0 []
    |  Data/Moby-Dick.txt HadoopRDD[6] at textFile at NativeMethodAccessorImpl.java:0 []


| Execution plan   | RDD |  Comments |
| :---------------------------------------------------------------- | :------------: | :--- |
|`(2)_PythonRDD[6] at RDD at PythonRDD.scala:48 []`| **counts** | Final RDD|
|`_/__MapPartitionsRDD[5] at mapPartitions at PythonRDD.scala:436 []`| **---"---** |
|`_/__ShuffledRDD[4] at partitionBy at NativeMethodAccessorImpl.java:0 [`| **---"---** | RDD is partitioned by key |
|`_+-(2)_PairwiseRDD[3] at reduceByKey at <timed exec>:4 []`| **---"---** | Perform mapByKey |
|`____/__PythonRDD[2] at reduceByKey at <timed exec>:4 []`| **words, not_empty, key_values** | The result of  partitioning into words|
| | |  removing empties, and making into (word,1) pairs|
|`____/__../../Data/Moby-Dick.txt MapPartitionsRDD[1] at textFile at Nat`| **text_file** | The partitioned text |
|`____/__../../Data/Moby-Dick.txt HadoopRDD[0] at textFile at NativeMeth`| **---"---** | The text source |

## Execution
Finally we count the number of times each word has occured.
Now, finally, the Lazy execution model finally performs some actual work, which takes a significant amount of time.

In [7]:
%%time
## Run #1
Count=counts.count()  # Count = the number of different words
Sum=counts.map(lambda x:x[1]).reduce(lambda x,y:x+y) # 
print('Different words=%5.0f, total words=%6.0f, mean no. occurances per word=%4.2f'%(Count,Sum,float(Sum)/Count))

Different words=33781, total words=215133, mean no. occurances per word=6.37
CPU times: user 26.2 ms, sys: 3.4 ms, total: 29.6 ms
Wall time: 2.52 s


### Amortization
When the same commands are performed repeatedly on the same data, the execution time tends to decrease in later executions.

The cells below are identical to the one above, with one exception at `Run #3`

Observe that `Run #2` take much less time that `Run #1`. Even though no `cache()` was explicitly requested. The reason is that Spark caches (or materializes) `key_values`, before executing `reduceByKey()` because performng reduceByKey requires a shuffle, and a shuffle requires that the input RDD is materialized. In other words, sometime caching happens even if the programmer did not ask for it.

In [8]:
%%time
## Run #2
Count=counts.count()
Sum=counts.map(lambda x:x[1]).reduce(lambda x,y:x+y)
print('Different words=%5.0f, total words=%6.0f, mean no. occurances per word=%4.2f'%(Count,Sum,float(Sum)/Count))

Different words=33781, total words=215133, mean no. occurances per word=6.37
CPU times: user 25.1 ms, sys: 10.5 ms, total: 35.5 ms
Wall time: 787 ms


### Explicit Caching
In `Run #3` we explicitly ask for `counts` to be cached. This will reduce the execution time in the following run `Run #4` by a little bit, but not by much.

In [9]:
%%time
## Run #3, cache
Count=counts.cache().count()
Sum=counts.map(lambda x:x[1]).reduce(lambda x,y:x+y)
print('Different words=%5.0f, total words=%6.0f, mean no. occurances per word=%4.2f'%(Count,Sum,float(Sum)/Count))

Different words=33781, total words=215133, mean no. occurances per word=6.37
CPU times: user 33.7 ms, sys: 2.24 ms, total: 36 ms
Wall time: 924 ms


In [10]:
%%time
#Run #4
Count=counts.count()
Sum=counts.map(lambda x:x[1]).reduce(lambda x,y:x+y)
print('Different words=%5.0f, total words=%6.0f, mean no. occurances per word=%4.2f'%(Count,Sum,float(Sum)/Count))

Different words=33781, total words=215133, mean no. occurances per word=6.37
CPU times: user 18.4 ms, sys: 7.6 ms, total: 26 ms
Wall time: 602 ms


In [11]:
%%time
#Run #5
Count=counts.count()
Sum=counts.map(lambda x:x[1]).reduce(lambda x,y:x+y)
print('Different words=%5.0f, total words=%6.0f, mean no. occurances per word=%4.2f'%(Count,Sum,float(Sum)/Count))

Different words=33781, total words=215133, mean no. occurances per word=6.37
CPU times: user 15.1 ms, sys: 9.96 ms, total: 25 ms
Wall time: 489 ms
